# NRC-23 - Image Quality Verification by Filter   

## Notebook: Compare empirical PSF with synthetic PSF from WebbPSF (single PSF)

**Author**: Matteo Correnti, STScI Scientist II
<br>
**Created**: December, 2021
<br>
**Last Updated**: February, 2022

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
    2.2 [Plotting functions imports](#matpl_imports)<br>
    2.3 [Select detector and filter](#sel_det)<br>
3. [Load empirical and synthetic PSFs](#load_psf)<br>
4. [Derive the radial profiles](#radial_profiles)<br>
    4.1 [Determine the center of the PSF](#find_center)<br>
    4.2 [Calculate radial profile - Method 1](#rp_method1)<br>
    4.3 [Display radial profiles comparison- Method 1](#plot_method1)<br>
    4.4 [Calculate radial profile - Method 2](#rp_method2)<br>
    4.5 [Display radial profiles comparison- Method 1](#plot_method2)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

This notebook shows how to derive the radial profile of a PSF, in order to compare the empirical PSF obtained from the observations with the synthetic PSF obtained from WebbPSF. In this case, we compare a single PSF with the ePSF obtained from different dithers (same detector - same filter).

**Dependencies**: before running this notebook it is necessary to create the synthetic model PSFs using the notebook `NRC-23_webbpsf.ipynb` and the empirical PSFs using the notebook `NRC-23_create_ePSF.ipynb` or created with different codes. 

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all the necessary Python packages and we define some plotting parameters.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import os 

os.environ['WEBBPSF_PATH'] = '/grp/jwst/ote/webbpsf-data'
os.environ['PYSIN_CDBS'] = '/grp/hst/cdbs/'

import glob as glob

import numpy as np

from astropy.io import fits
from astropy.stats import sigma_clipped_stats

import webbpsf
from webbpsf.utils import to_griddedpsfmodel

import pysynphot

from photutils.centroids import centroid_2dg, centroid_sources, centroid_com
from photutils import CircularAperture, CircularAnnulus, aperture_photometry


### 2.2<font color='white'>-</font>Plotting function imports<a class="anchor" id="matpl_imports"></a> ###

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 20

font1 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '12'}
font2 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '20'}

In [ ]:
figures_dir = 'FIGURES/'

if not os.path.exists(figures_dir):
    os.makedirs(figures_dir)

### 2.3<font color='white'>-</font>Select detector and filter<a class="anchor" id="sel_det"></a> ###

In [ ]:
det = 'NRCB1'
filt = 'F200W'

3.<font color='white'>-</font>Load empirical and synthetic PSFs<a class="anchor" id="load_psf"></a>
------------------

In [ ]:
distorted = True
fov = 41
oversample = 4
num_psf = 1

if distorted:

    dir_webbpsf = 'PSF_MODELS/Distorted/Fov{}px_numPSFs{}_oversample{}'.format(fov, num_psf, oversample)
    dir_epsf = 'ePSF_MODELS/Distorted/Single_PSF/Fov{}px_numPSFs{}_oversample{}'.format(fov, num_psf, oversample)
    
    
    epsfs = sorted(glob.glob(os.path.join(dir_epsf, '*'+det+'_'+filt+'*')))
    psf_webbpsf = glob.glob(os.path.join(dir_webbpsf, '*'+filt+'*'+str.lower(det)+'*'))[0]

else:
    
    dir_webbpsf = 'PSF_MODELS/Undistorted/Fov{}px_numPSFs{}_oversample{}'.format(fov, num_psf, oversample)
    dir_epsf = 'ePSF_MODELS/Undistorted/Fov{}px_numPSFs{}_oversample{}'.format(fov, num_psf, oversample)
    epsfs = sorted(glob.glob(os.path.join(dir_epsf, '*'+det+'_'+filt+'*')))
    psf_webbpsf = glob.glob(os.path.join(dir_webbpsf, '*'+filt+'*'+str.lower(det)+'*'))[0]

print('Empirical PSFs:', epsfs)
print('Synthetic PSFs from WebbPSF:', psf_webbpsf)

4.<font color='white'>-</font>Derive the radial profiles<a class="anchor" id="radial_profiles"></a>
------------------

We adopt two different methods to calculate the radial profiles:

* Method 1: from aperture photometry at increasingly larger radii
* Method 2: adapted version of the WebbPSF function `radial_profile` 

For the original version of the `radial_profile` code see [here](https://github.com/spacetelescope/poppy/blob/develop/poppy/utils.py). 

More information on aperture photometry using Photutils can be found here: [Aperture Photometry](https://photutils.readthedocs.io/en/stable/aperture.html). In the function below, to calculate the radial profile, we first determine the aperture photometry at different radii, then we derive the flux in each annulus and divide by the appropriate area. 

The main difference is that with aperture photometry the exact intersection of the aperture with each pixel is calculated.

### 4.1<font color='white'>-</font>Determine the center of the PSF<a class="anchor" id="find_center"></a> ###

In [ ]:
def find_center(data, centroid_func=centroid_com):

    xcenter = data.shape[1] / 2
    ycenter = data.shape[0] / 2
    
    xc, yc = centroid_sources(data, xcenter, ycenter, box_size=21, centroid_func=centroid_func)
    
    return xc, yc

### 4.2<font color='white'>-</font>Calculate radial profile - Method 1<a class="anchor" id="rp_method1"></a> ###

In [ ]:
def ap_phot(data, positions, radii, rsky):
    
    zero_mask = np.where(data == 0,0,1)
    nan_mask  = np.where(np.isnan(data),0,1)
    zero_mask = nan_mask * zero_mask
    
    nan_mask = np.where(zero_mask == 0,True,False)

    apflux = []
    apflux_err = []
    area = []
     
    for r in radii:
        
        aperture = CircularAperture(positions, r=r)
        aperture_area = aperture.area

        annulus_aperture = CircularAnnulus(positions, r_in=rsky[0], r_out=rsky[1])
        annulus_mask = annulus_aperture.to_mask(method='center')
        
        local_sky_median = []
        local_sky_stdev = []
        
        annulus_data = annulus_mask.multiply(data)
        annulus_data_1d = annulus_data[annulus_mask.data > 0]
        
        annulus_data_1d = annulus_data_1d[np.isfinite(annulus_data_1d)]
            
        _, median_sigclip, stdev_sigclip = sigma_clipped_stats(annulus_data_1d)
        
        local_sky_median.append(median_sigclip)
        local_sky_median = np.array(local_sky_median)
        local_sky_stdev.append(stdev_sigclip)
        local_sky_stdev = np.array(local_sky_stdev)
       
        phot = aperture_photometry(data, aperture, method='exact', mask=nan_mask)
        
        phot['aper_bkg'] = local_sky_median * aperture.area
        phot['aper_sum_bkgsub'] = phot['aperture_sum'] - phot['aper_bkg']
        
        aperture_sum = phot['aper_sum_bkgsub'][0]
    
        error_poisson = phot['aperture_sum']
        error_scatter_sky = aperture.area * local_sky_stdev**2
        error_mean_sky = local_sky_stdev**2 * aperture.area**2 / annulus_aperture.area
        
        ap_err = np.sqrt(error_poisson + error_scatter_sky + error_mean_sky)
         
        apflux.append(aperture_sum)
        apflux_err.append(ap_err)
        area.append(aperture_area)

    return apflux, area

In [ ]:
def calc_profile(image, model='webbpsf', norm=True, peak=True):
    
    if model=='webbpsf':
        
        im = to_griddedpsfmodel(image)
        data = im.data[0]
        fov = data.shape[1]
    
    if model=='epsf':
        
        im = fits.open(image)
        data = im[0].data
        fov = data.shape[1] - 1
    
    xc, yc = find_center(data, centroid_func=centroid_com)

    positions = (xc[0], yc[0])
    print('Center calcuated with centroid function: X =', positions[0], 'Y =', positions[1])
    
    radii = np.arange(2, data.shape[1]/2 -6, 2)
    rsky = [data.shape[1]/2 - 5, data.shape[1]/2 - 1]
    
    apflux, area = ap_phot(data, positions, radii, rsky)
    
    diff = np.zeros(len(apflux))
    encircled = np.zeros(len(apflux))
    
    diff[0] = apflux[0]
    encircled[0] = apflux[0] / area[0]
    
    sum_ee = encircled[0]
    
    ii = 0 
    
    for ii in range(1, len(apflux)):
        diff[ii] = apflux[ii] - apflux[ii - 1]
        encircled[ii] = diff[ii] / (area[ii] - area[ii - 1])
        sum_ee = sum_ee + encircled[ii]
        
    if norm:
        diff = diff / diff[0]
        
        if peak:
            
            encircled = encircled / encircled[0]
        else:
            
            encircled = encircled / sum_ee
    
    return diff, encircled, radii, fov
    

In [ ]:
diff_webbpsf, en1_webbpsf, radii_webbpsf, fov_webbpsf = calc_profile(psf_webbpsf, model='webbpsf', norm=True, peak=True)
_ , en2_webbpsf, _, _ = calc_profile(psf_webbpsf, model='webbpsf', norm=True, peak=False)


In [ ]:
diff_epsf_tot = []
en1_epsf_tot = []
en2_epsf_tot = []

for epsf in epsfs:
    
    diff_epsf, en1_epsf, radii_epsf, _ = calc_profile(epsf, model='epsf', norm=True, peak=True)
    _ , en2_epsf, _, _ = calc_profile(epsf, model='epsf', norm=True, peak=False)
    
    diff_epsf_tot.append(diff_epsf)
    en1_epsf_tot.append(en1_epsf)
    en2_epsf_tot.append(en2_epsf)


In [ ]:
def pix2arcsec_sw(x):
    return x * 0.031
def arcsec2pix_sw(x):
    return x / 0.031
def pix2arcsec_lw(x):
    return x * 0.063
def arcsec2pix_lw(x):
    return x / 0.063

### 4.3<font color='white'>-</font>Display radial profiles comparison - Method 1<a class="anchor" id="plot_method1"></a> ###

In [ ]:
oversample = 4
rad_arcsec_webbpsf = pix2arcsec_sw(radii_webbpsf / 4)
rad_arcsec_epsf = pix2arcsec_sw(radii_epsf / 4)
fov_detsamp = int(fov_webbpsf / 4)

plt.figure(figsize=(14,8))

plt.suptitle(det+' - '+ filt, fontsize=30)

ax1 = plt.subplot(1, 2, 1)

xlim0 = rad_arcsec_epsf[0] - 0.02
xlim1 = np.max(rad_arcsec_epsf) + 0.01

ax1.set_xlim(xlim0, xlim1)

ax1.set_xlabel('radius [arcsec]', fontdict=font2)
ax1.set_ylabel('Normalized surface brightess profile (flux/arcsec^2)', fontdict=font2)
    
ax1.xaxis.set_major_locator(ticker.AutoLocator())
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.scatter(rad_arcsec_webbpsf, en1_webbpsf, s=50, c='k')
ax1.semilogy(rad_arcsec_webbpsf, en1_webbpsf, c='k')

for j in en1_epsf_tot:   
    ax1.scatter(rad_arcsec_epsf, j, s=50)
    ax1.semilogy(rad_arcsec_epsf, j)

ax1b = ax1.secondary_xaxis('top', functions=(arcsec2pix_sw, pix2arcsec_sw))
ax1b.set_xlabel('radius [px]', fontdict=font2)

ax2 = plt.subplot(1, 2, 2)

ax2.set_xlim(xlim0, xlim1)

ax2.set_xlabel('radius [arcsec]', fontdict=font2)
ax2.set_ylabel('Normalized surface brightess profile (flux/arcsec^2)', fontdict=font2)
    
ax2.xaxis.set_major_locator(ticker.AutoLocator())
ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax2.yaxis.set_major_locator(ticker.AutoLocator())
ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax2.scatter(rad_arcsec_webbpsf, en2_webbpsf, s=50, c='k')
ax2.semilogy(rad_arcsec_webbpsf, en2_webbpsf, c='k')

for j in en2_epsf_tot:   
    ax2.scatter(rad_arcsec_epsf, j, s=50)
    ax2.semilogy(rad_arcsec_epsf, j)

ax2b = ax2.secondary_xaxis('top', functions=(arcsec2pix_sw, pix2arcsec_sw))
ax2b.set_xlabel('radius [px]', fontdict=font2)

plt.tight_layout()

filename = 'radial_profile_{}_{}_fov{}px.png'.format(det, filt, fov_detsamp)
    
#plt.savefig(os.path.join(figures_dir, filename))

### 4.4<font color='white'>-</font>Calculate radial profile - Method 2<a class="anchor" id="rp_method2"></a> ###

In [ ]:
def calc_rp(image=None, model='epsf', center=False, pixelscale=0.031, oversample=4, binsize=None, 
            maxradius=None, norm='peak'):
    
    if model=='epsf':
        
        im = fits.open(image)
        data = im[0].data
        
        
    if model=='webbpsf':
        
        im = to_griddedpsfmodel(image)
        data = im.data[0]
        
        
    y,x = np.indices(data.shape, dtype=float)
    
    if center:
        xc, yc = find_center(data, centroid_func=centroid_com)
    
        positions = (xc[0], yc[0])
        print ('Center calculated with centroid function: X =', positions[0], 'Y =', positions[1])
         
    else:
        max_image = np.max(data)
        center = np.argwhere(data == max_image)
        
        print('XY position of the PSF maximum. X = ', center[0][1], 'Y =', center[0][0])
    
    x -= positions[0]#center[0][1]
    y -= positions[1]#center[0][0]
    
    data_norm = data.copy()
    
    if norm=='peak':
    
        data_norm /= data_norm.max()
    
    if norm=='total':
        data_norm /= data_norm.sum()
    
    pxscl = pixelscale / oversample
    
    if binsize is None:
        binsize = pxscl
   
    r = np.sqrt(x**2 + y**2) * pxscl / binsize
    ind = np.argsort(r.flat)
    sr = r.flat[ind] 
    sim = data_norm.flat[ind]
    ri = sr.astype(int) 
    deltar = ri[1:] - ri[:-1]  
    rind = np.where(deltar)[0]
    nr = rind[1:] - rind[:-1]  
    csim = np.nan_to_num(sim).cumsum(dtype=float)  
    tbin = csim[rind[1:]] - csim[rind[:-1]]  
    radialprofile = tbin / nr
    radialprofile2 = np.empty(len(radialprofile) + 1)
    if rind[0] != 0:
        radialprofile2[0] = csim[rind[0]] / (
                rind[0] + 1)  
    else:
        radialprofile2[0] = csim[0]  
    radialprofile2[1:] = radialprofile

    rr = np.arange(ri.min(), ri.min() + len(
        radialprofile2)) * binsize + binsize * 0.5  # these should be centered in the bins, so add a half.
    
    if maxradius is not None:
        
        maxrad = maxradius * pxscl
        crop = rr < maxrad
        rr = rr[crop]
        radialprofile2 = radialprofile2[crop]
    
    
    return rr, radialprofile2

In [ ]:
r_webbpsf, rp_webbpsf = calc_rp(image=psf_webbpsf, center=True, model='webbpsf', maxradius=70, norm='total')
print(len(rp_webbpsf))
rs = []
rps = []

for epsf in epsfs:
    r, rp = calc_rp(image=epsf, center=True, model='epsf', norm='total')
    rs.append(r)  
    rps.append(rp)

### 4.5<font color='white'>-</font>Display radial profiles comparison - Method 2<a class="anchor" id="plot_method2"></a> ###

In [ ]:
plt.figure(figsize=(12,10))

ax = plt.subplot(1, 1, 1)
        
plt.suptitle(det+' - '+ filt, fontsize=30)
    
xlim0 = np.min(r_webbpsf)
xlim1 = np.max(r_webbpsf)

ax.set_xlim(xlim0, xlim1)
    
ax.set_xlabel('radius [arcsec]', fontdict=font2)
ax.set_ylabel('Normalized surface brightess profile (flux/arcsec^2)', fontdict=font2)
    
ax.xaxis.set_major_locator(ticker.AutoLocator())
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_major_locator(ticker.AutoLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
   
ax.semilogy(r_webbpsf, rp_webbpsf, c='k')
ax.scatter(r_webbpsf, rp_webbpsf, s=50, c='k')

for i,j in zip(rs, rps):   
    ax.scatter(i, j, s=50)
    ax.semilogy(i, j)

ax1b = ax.secondary_xaxis('top', functions=(arcsec2pix_sw, pix2arcsec_sw))
ax1b.set_xlabel('radius [px]', fontdict=font2)

plt.tight_layout()

filename = 'radial_profile_webbpsf_{}_{}_fov{}px.png'.format(det, filt, fov_detsamp)
    
#plt.savefig(os.path.join(figures_dir, filename))